<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FP_AGENT_AUTOGEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries for AutoGen (and dependencies like openai)
!pip install autogen -q # Specify a version if needed
!pip install openai -q
!pip install colab-env -q
!pip install litellm -q
import colab_env

## openai

https://platform.openai.com/settings/organization/billing/overview

In [4]:
import os
from autogen import Agent, UserProxyAgent, config_list_from_json, ConversableAgent
from openai import OpenAI # Import OpenAI client for explicit use if needed, though AutoGen handles it via config

# Get OpenAI API key (still needed for gpt models)
try:
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
except ImportError:
    openai_api_key = os.getenv('OPENAI_API_KEY')

if not openai_api_key:
    print("WARNING: OPENAI_API_KEY not found. gpt-4 model may not be available.")


# 2. Define Tool Functions (AutoGen style)
# They are regular Python functions that AutoGen agents can call via UserProxyAgent.

def search_aviation_database(query: str) -> str:
    """Searches aviation databases for NOTAMs, airport info, airspace, weather, maintenance, or passenger data."""
    print(f"\n--- Tool Call: search_aviation_database with query: '{query}' ---")
    # Implement the logic from AviationDatabaseSearchTool._run here
    if "NOTAMs" in query:
        return f"Aviation database search results for '{query}': NOTAMs indicate temporary flight restriction near XYZ airport from 1400Z-1800Z due to airshow. (Simulated)"
    elif "airport information" in query:
        return f"Aviation database search results for '{query}': KORD: Runways 09L/27R, 09R/27L active. EDDF: Runways 07L/25R, 07R/25L active. Fuel available at both. (Simulated)"
    elif "weather" in query:
        # Simulate a relevant weather report for the NAT selection
        return f"Aviation database search results for '{query}': KORD METAR: 062200Z 27010KT 10SM BKN050 18/08 A2995. EDDF METAR: 070400Z 23005KT 9999 FEW020 15/10 Q1012. Significant en-route weather: Moderate turbulence over Greenland, strong jet stream at FL350-FL390 with winds up to 150 kts, strongest favoring a more northerly track. (Simulated)"
    elif "maintenance records for Boeing 777-300ER Tail N777UA June 7 2025" in query:
        return f"Aviation database search results for '{query}': Boeing 777-300ER (Tail N777UA) last major inspection (C-check) completed on 2025-05-15. No deferred defects relevant to flight payload. (Simulated)"
    elif "passenger manifest for KORD-EDDF flight June 7 2025" in query:
        return f"Aviation database search results for '{query}': Estimated 250 adult passengers and 20 children for KORD-EDDF flight. Average adult weight 190 lbs, average child weight 80 lbs. Estimated 15,000 lbs of baggage. (Simulated)"
    else:
        return f"Aviation database search results for '{query}': No specific details found, general aviation data available. (Simulated)"

def query_air_traffic_control(query: str) -> str:
    """Queries current air traffic control instructions, clearances, and real-time traffic data, including dynamic North Atlantic Tracks (NATs) and their waypoints."""
    print(f"\n--- Tool Call: query_air_traffic_control with query: '{query}' ---")
    # Implement the logic from AirTrafficControlTool._run here
    if "NATs for eastbound traffic June 7 2025" in query:
        nat_data = {
            "NAT A": "ENTRY: N50W020 EXIT: N52W050 (Waypoints: 50N/020W, 50N/030W, 51N/040W, 52N/050W) - More southerly, might face headwind.",
            "NAT B": "ENTRY: N49W015 EXIT: N51N055 (Waypoints: 49N/015W, 49N/025W, 50N/035W, 51N/045W, 51N/055W) - Balanced track.",
            "NAT C": "ENTRY: N48W010 EXIT: N50W060 (Waypoints: 48N/010W, 48N/020W, 49N/030W, 49N/040W, 50N/050W, 50N/060W) - More northerly, currently offers best tailwinds.",
            "Recommended Eastbound (considering simulated winds)": "NAT C - Offering most favorable winds and efficient routing for KORD-EDDF."
        }
        return f"Air Traffic Control data for '{query}': {nat_data} (Simulated based on typical NAT structure and wind preference)"
    elif "oceanic clearance" in query:
        return f"Air Traffic Control data for '{query}': Current clearances suggest a routing via oceanic entry point DOGAL, altitude FL370. Expect oceanic clearance 30 mins prior to entry. (Simulated)"
    return f"Air Traffic Control data for '{query}': No specific ATC details found for query. (Simulated)"

# 3. Define Configuration List for AutoGen Agents

try:

    # Define the config list using the OpenAI API key only
    config_list = [
         {
            "model": 'gpt-4', # Add the GPT-4 model
            "api_key": openai_api_key,
         },
         {
            "model": 'gpt-4-turbo-preview', # You could also use a newer version if available
            "api_key": openai_api_key,
         },
    ]

    # Filter out entries if the API key is missing
    config_list = [
        config for config in config_list if config.get("api_key") is not None
    ]

    print(f"AutoGen config_list defined with {len(config_list)} available models.")

except Exception as e:
    print(f"An unexpected error occurred setting up config_list: {e}")
    print("Using a minimal fallback (may not work).")
    config_list = []


# 4. Define Agents in AutoGen
# UserProxyAgent for tool execution and human interaction (optional)
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A helpful administrator who can run tool functions requested by other agents. Respond 'TERMINATE' when the flight plan is fully generated and complete.",
    human_input_mode="NEVER", # Set to ALWAYS or AS_NEEDED to observe/guide
    max_consecutive_auto_reply=20, # Increased limit for potentially longer conversations
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").upper(), # Case-insensitive terminate
    function_map={
        "search_aviation_database": search_aviation_database,
        "query_air_traffic_control": query_air_traffic_control,
        # Removed "ask_deepseek_flight_ai" from the function map
    },
     # Add llm_config if this agent needs to use an LLM itself (e.g., to decide which tool to call)
    llm_config={"config_list": config_list} # Use the correctly configured list
)

# Conversable Agents for each role
# Ensure llm_config uses the correctly configured list
flight_planner_agent = ConversableAgent(
    name="FlightPlanner",
    system_message="""You are an Expert Flight Plan Creator. Your goal is to develop optimized and safe flight plans,
    considering aviation regulations, weather, and aircraft performance, with a strong emphasis on fuel efficiency.
    Collaborate with other agents (WeatherAnalyst, AirspaceExpert, PerformanceSpecialist, MaintenanceCoordinator, PassengerCoordinator, ContingencyPlanner)
    to gather necessary information. You need the following information to create the plan:
    1. Weather briefing, including winds/jet stream for NAT selection and altitude recommendations.
    2. Relevant airspace restrictions and NOTAMs.
    3. Aircraft maintenance status and limitations.
    4. Estimated passenger and baggage payload.
    5. Optimal performance parameters (altitudes, speeds, fuel burn) based on weather and payload.
    6. Contingency plan with alternate airports.

    Once you have ALL this information from the other agents, synthesize it to create a detailed flight plan
    for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
    Include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
    chosen fuel-efficient North Atlantic Track (NAT) with waypoints and entry/exit points,
    recommended step climbs/Mach/airspeed, and at least two alternate airports.
    Format the final flight plan clearly in markdown.
    When the flight plan is complete and includes all requested details, state 'TERMINATE' at the end of your message.
    If you need information, clearly state what you need and which agent should provide it.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    # Auto reply set to False to allow the GroupChat to manage turns more
    # auto_reply_agent=None
)

weather_forecaster_agent = ConversableAgent(
    name="WeatherAnalyst",
    system_message="""You are an Aviation Weather Analyst. Provide detailed and accurate aviation weather forecasts for the KORD to EDDF flight on June 7, 2025.
    Identify potential hazards along the flight route. **Explicitly analyze wind components (jet stream) to recommend the most fuel-efficient North Atlantic Track and optimal cruising altitudes.**
    Request necessary weather data using the 'search_aviation_database' tool via the Admin.
    Once you have the weather data, provide a concise summary to the FlightPlanner, clearly stating your NAT and altitude recommendations based on winds.""",
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

airspace_regulator_agent = ConversableAgent(
    name="AirspaceExpert",
    system_message="""You are an Airspace and Regulatory Compliance Expert. Identify and list all airspace restrictions, NOTAMs,
    and special use airspaces that impact the flight route from KORD to EDDF, including oceanic airspace procedures (e.g., NATs).
    Ensure any proposed route or maneuvers comply with all regulations.
    Request necessary data using the 'search_aviation_database' and 'query_air_traffic_control' tools via the Admin.
    Provide a list of relevant regulations and NOTAMs to the FlightPlanner.""",
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

aircraft_performance_analyst_agent = ConversableAgent(
    name="PerformanceSpecialist",
     system_message="""You are an Aircraft Performance Specialist for a Boeing 777-300ER. Based on the determined payload (from PassengerCoordinator/MaintenanceCoordinator)
     and wind data (from WeatherAnalyst), **calculate the optimal cruising altitude(s) for maximum fuel efficiency, considering the latest wind data (jet stream).
     Recommend specific Mach numbers or airspeeds for different flight segments (climb, cruise, descent) to minimize fuel burn.** Also provide the estimated fuel burn for the transatlantic primary route.
     You may use available tools via the Admin for calculations and data retrieval.
     Provide your payload calculation, optimal performance parameters, and estimated fuel burn clearly to the FlightPlanner.""",
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

contingency_planner_agent = ConversableAgent(
    name="ContingencyPlanner",
    system_message="""You are an Emergency and Contingency Flight Planner. Develop a robust contingency plan for the KORD to EDDF flight.
    Include at least three suitable alternate airports (e.g., Gander, Reykjavik, Shannon, Amsterdam) and basic diversion procedures for a transatlantic crossing.
    **Include considerations for fuel-efficient diversion profiles if an alternate landing is required.**
    Use tools via the Admin if needed to find airport information or weather for alternates.
    Provide the contingency plan to the FlightPlanner.""",
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

maintenance_coordinator_agent = ConversableAgent(
    name="MaintenanceCoordinator",
    system_message="""You are an Aircraft Maintenance and Readiness Coordinator for a Boeing 777-300ER (Tail N777UA).
    Assess aircraft maintenance status and identify any relevant limitations or weight considerations for the flight on June 7, 2025.
    Use the 'search_aviation_database' tool via the Admin to get maintenance records.
    Report any limitations or weight impacts to the PerformanceSpecialist and FlightPlanner.""",
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

passenger_coordinator_agent = ConversableAgent(
    name="PassengerCoordinator",
    system_message="""You are a Passenger and Cargo Load Estimator. Estimate the total weight of passengers, baggage, and cargo
    for the KORD-EDDF flight on June 7, 2025, to determine the flight payload.
    Use the 'search_aviation_database' tool via the Admin to get passenger manifest information.
    Provide the estimated number of adult/child passengers, estimated baggage/cargo weight, and the total estimated payload (in lbs or kg)
    to the PerformanceSpecialist and FlightPlanner.
    """,
    llm_config={"config_list": config_list}, # Use the correctly configured list
    human_input_mode="NEVER",
)

# 5. Create and Initiate the Group Chat
# Using GroupChat helps manage the conversation flow between multiple agents.
from autogen import GroupChat, GroupChatManager

groupchat = GroupChat(
    agents=[
        user_proxy,
        flight_planner_agent,
        weather_forecaster_agent,
        airspace_regulator_agent,
        aircraft_performance_analyst_agent,
        contingency_planner_agent,
        maintenance_coordinator_agent,
        passenger_coordinator_agent
        ],
    messages=[],
    max_round=30, # Increase max rounds for complex tasks
    speaker_selection_method="auto", # 'auto' lets the LLM decide who speaks next
    allow_repeat_speaker=False # Prevent the same agent from talking twice in a row
)

manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list}) # Use the correctly configured list

print("Starting the AutoGen Flight Planning Collaboration with a focus on fuel efficiency...")

# Initiate the chat with the manager, providing the initial task description.
user_proxy.initiate_chat(
    manager,
    message="""Generate a detailed, fuel-efficient flight plan for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
    The plan must include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
    the most fuel-efficient North Atlantic Track (NAT) for eastbound traffic on June 7, 2025 (including all oceanic waypoints, entry/exit points),
    recommended step climbs/Mach/airspeed for fuel efficiency, and at least two alternate airports.
    Collaborate as a team to gather all necessary information (weather, regulations, maintenance, payload) and calculate performance before the FlightPlanner generates the final output.
    The final output should be a markdown formatted document.
    Ensure the final message clearly states 'TERMINATE' at the end."""
)

print("\n\n########################")
print("## AutoGen Conversation Finished ##")
print("########################\n")

AutoGen config_list defined with 2 available models.
Starting the AutoGen Flight Planning Collaboration with a focus on fuel efficiency...
Admin (to chat_manager):

Generate a detailed, fuel-efficient flight plan for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
    The plan must include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
    the most fuel-efficient North Atlantic Track (NAT) for eastbound traffic on June 7, 2025 (including all oceanic waypoints, entry/exit points),
    recommended step climbs/Mach/airspeed for fuel efficiency, and at least two alternate airports.
    Collaborate as a team to gather all necessary information (weather, regulations, maintenance, payload) and calculate performance before the FlightPlanner generates the final output.
    The final output should be a markdown formatted document.
    Ensure the final message clearly states 'TERMINATE' at the end.

---------------------------------------

## deepseek

https://platform.deepseek.com/usage

In [2]:
import os
# Import OpenAI client, as DeepSeek's API is compatible with it
from openai import OpenAI
from autogen import Agent, UserProxyAgent, config_list_from_json, ConversableAgent


# 1. Get API Keys
# Removed code for openai_api_key

# Get DeepSeek API Key
deepseek_api_key = None # Initialize as None
try:
    # Assuming you store your DeepSeek key as 'DEEPSEEK_API_KEY'
    from google.colab import userdata
    deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
    print("Attempted to get DeepSeek API key from Colab userdata.")
except ImportError:
    print("Not in Colab environment or userdata module not available.")
    pass # Ignore ImportError if not in Colab environment

# If key not found in userdata (or not in Colab), try environment variables
if deepseek_api_key is None:
    deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
    if deepseek_api_key is not None:
        print("Attempted to get DeepSeek API key from environment variables.")

# Print a masked version of the key for verification
if deepseek_api_key:
    print(f"DeepSeek API Key successfully retrieved (starts with: {deepseek_api_key[:4]}...)")
else:
    print("DeepSeek API Key not found in Colab userdata or environment variables.")


# 2. Define Tool Functions (AutoGen style)
# These tools simulate external systems and don't directly depend on DeepSeek,
# but agents might use DeepSeek to decide when/how to call them.

def search_aviation_database(query: str) -> str:
    """Searches aviation databases for NOTAMs, airport info, airspace, weather, maintenance, or passenger data."""
    print(f"\n--- Tool Call: search_aviation_database with query: '{query}' ---")
    if "NOTAMs" in query:
        return f"Aviation database search results for '{query}': NOTAMs indicate temporary flight restriction near XYZ airport from 1400Z-1800Z due to airshow. (Simulated)"
    elif "airport information" in query:
        return f"Aviation database search results for '{query}': KORD: Runways 09L/27R, 09R/27L active. EDDF: Runways 07L/25R, 07R/25L active. Fuel available at both. (Simulated)"
    elif "weather" in query:
        return f"Aviation database search results for '{query}': KORD METAR: 062200Z 27010KT 10SM BKN050 18/08 A2995. EDDF METAR: 070400Z 23005KT 9999 FEW020 15/10 Q1012. Significant en-route weather: Moderate turbulence over Greenland, strong jet stream at FL350-FL390 with winds up to 150 kts, strongest favoring a more northerly track. (Simulated)"
    elif "maintenance records for Boeing 777-300ER Tail N777UA June 7 2025" in query:
        return f"Aviation database search results for '{query}': Boeing 777-300ER (Tail N777UA) last major inspection (C-check) completed on 2025-05-15. No deferred defects relevant to flight payload. (Simulated)"
    elif "passenger manifest for KORD-EDDF flight June 7 2025" in query:
        return f"Aviation database search results for '{query}': Estimated 250 adult passengers and 20 children for KORD-EDDF flight. Average adult weight 190 lbs, average child weight 80 lbs. Estimated 15,000 lbs of baggage. (Simulated)"
    else:
        return f"Aviation database search results for '{query}': No specific details found, general aviation data available. (Simulated)"

def query_air_traffic_control(query: str) -> str:
    """Queries current air traffic control instructions, clearances, and real-time traffic data, including dynamic North Atlantic Tracks (NATs) and their waypoints."""
    print(f"\n--- Tool Call: query_air_traffic_control with query: '{query}' ---")
    if "NATs for eastbound traffic June 7 2025" in query:
        nat_data = {
            "NAT A": "ENTRY: N50W020 EXIT: N52W050 (Waypoints: 50N/020W, 50N/030W, 51N/040W, 52N/050W) - More southerly, might face headwind.",
            "NAT B": "ENTRY: N49W015 EXIT: N51N055 (Waypoints: 49N/015W, 49N/025W, 50N/035W, 51N/045W, 51N/055W) - Balanced track.",
            "NAT C": "ENTRY: N48W010 EXIT: N50W060 (Waypoints: 48N/010W, 48N/020W, 49N/030W, 49N/040W, 50N/050W, 50N/060W) - More northerly, currently offers best tailwinds.",
            "Recommended Eastbound (considering simulated winds)": "NAT C - Offering most favorable winds and efficient routing for KORD-EDDF."
        }
        return f"Air Traffic Control data for '{query}': {nat_data} (Simulated based on typical NAT structure and wind preference)"
    elif "oceanic clearance" in query:
        return f"Air Traffic Control data for '{query}': Current clearances suggest a routing via oceanic entry point DOGAL, altitude FL370. Expect oceanic clearance 30 mins prior to entry. (Simulated)"
    return f"Air Traffic Control data for '{query}': No specific ATC details found for query. (Simulated)"

# Define the ask_deepseek_flight_ai tool function
def ask_deepseek_flight_ai(prompt: str, model: str = 'deepseek-reasoner') -> str:
    """Uses the DeepSeek AI model for complex reasoning and response generation on aviation topics."""
    print(f"\n--- Tool Call: ask_deepseek_flight_ai with model '{model}' and prompt: '{prompt[:100]}...' ---")
    if not deepseek_api_key:
        # This case should ideally be caught by the config_list check,
        # but good defensive coding.
        return "Error: DeepSeek API key not available for tool execution."
    try:
        # Use the deepseek_api_key
        client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant specializing in flight planning and aviation."},
                {"role": "user", "content": prompt},
            ],
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error interacting with DeepSeek API: {e}"


# 3. Define Configuration List for AutoGen Agents - DeepSeek Only

config_list = [] # Initialize config_list outside the try block
try:
    # Add DeepSeek models only if API key is available
    if deepseek_api_key:
         config_list.append({
            "model": 'deepseek/deepseek-reasoner',
            "api_key": deepseek_api_key,
            "base_url": "https://api.deepseek.com",
            "api_type": "openai",
         })
         config_list.append({
            "model": 'deepseek-coder',
            "api_key": deepseek_api_key,
            "base_url": "https://api.deepseek.com/v1",
            "api_type": "openai",
            # Optional: Add price here if you know it
            # "price": [0.002, 0.002] # Example prices
         })
         config_list.append({
            "model": 'deepseek-chat',
            "api_key": deepseek_api_key,
            "base_url": "https://api.deepseek.com/v1",
            "api_type": "openai",
            # Optional: Add price here if you know it
            # "price": [0.001, 0.001] # Example prices
         })
         # Add any other DeepSeek models you want to use here

except Exception as e:
    print(f"An unexpected error occurred while building config_list: {e}")
    # config_list remains [] in case of error

# Print the final config_list to verify its contents
#print("\n--- Final AutoGen config_list ---")
#print(config_list)
#print("---------------------------------\n")


# Check if any DeepSeek models were configured. If not, exit gracefully.
if not config_list:
    print("FATAL ERROR: AutoGen config_list is empty. No LLM models are configured.")
    print("Please ensure DEEPSEEK_API_KEY is set correctly.")
    # Use sys.exit() or raise an exception to stop execution if models are mandatory
    # In a Jupyter environment, raising an error might be cleaner than sys.exit()
    raise ValueError("No LLM models configured due to missing API key.")
    # If you don't want to stop execution, comment out the raise line
    # and understand that the agents will likely fail to interact.


# 4. Define Agents in AutoGen
# UserProxyAgent for tool execution and human interaction (optional)
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A helpful administrator who can run tool functions requested by other agents. Respond 'TERMINATE' when the flight plan is fully generated and complete.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=20,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").upper(),
    function_map={
        "search_aviation_database": search_aviation_database,
        "query_air_traffic_control": query_air_traffic_control,
        "ask_deepseek_flight_ai": ask_deepseek_flight_ai, # The DeepSeek tool function is still available
    },
     # This agent needs LLM access potentially to decide on tool calls or responses
    llm_config={"config_list": config_list} # Use the correctly configured list
)

# Conversable Agents for each role - configured to use the DeepSeek-only config_list
flight_planner_agent = ConversableAgent(
    name="FlightPlanner",
    system_message="""You are an Expert Flight Plan Creator. Your goal is to develop optimized and safe flight plans,
    considering aviation regulations, weather, and aircraft performance, with a strong emphasis on fuel efficiency.
    Collaborate with other agents (WeatherAnalyst, AirspaceExpert, PerformanceSpecialist, MaintenanceCoordinator, PassengerCoordinator, ContingencyPlanner)
    to gather necessary information. You need the following information to create the plan:
    1. Weather briefing, including winds/jet stream for NAT selection and altitude recommendations.
    2. Relevant airspace restrictions and NOTAMs.
    3. Aircraft maintenance status and limitations.
    4. Estimated passenger and baggage payload.
    5. Optimal performance parameters (altitudes, speeds, fuel burn) based on weather and payload.
    6. Contingency plan with alternate airports.

    Once you have ALL this information from the other agents, synthesize it to create a detailed flight plan
    for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
    Include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
    chosen fuel-efficient North Atlantic Track (NAT) with waypoints and entry/exit points,
    recommended step climbs/Mach/airspeed, and at least two alternate airports.
    Format the final flight plan clearly in markdown.
    When the flight plan is complete and includes all requested details, state 'TERMINATE' at the end of your message.
    If you need information, clearly state what you need and which agent should provide it.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

weather_forecaster_agent = ConversableAgent(
    name="WeatherAnalyst",
    system_message="""You are an Aviation Weather Analyst. Provide detailed and accurate aviation weather forecasts for the KORD to EDDF flight on June 7, 2025.
    Identify potential hazards along the flight route. **Explicitly analyze wind components (jet stream) to recommend the most fuel-efficient North Atlantic Track and optimal cruising altitudes.**
    Request necessary weather data using the 'search_aviation_database' tool via the Admin.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

airspace_regulator_agent = ConversableAgent(
    name="AirspaceExpert",
    system_message="""You are an Airspace and Regulatory Compliance Expert. Identify and list all airspace restrictions, NOTAMs,
    and special use airspaces that impact the flight route from KORD to EDDF, including oceanic airspace procedures (e.g., NATs).
    Ensure any proposed route or maneuvers comply with all regulations.
    Request necessary data using the 'search_aviation_database' and 'query_air_traffic_control' tools via the Admin.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

aircraft_performance_analyst_agent = ConversableAgent(
    name="PerformanceSpecialist",
     system_message="""You are an Aircraft Performance Specialist for a Boeing 777-300ER. Based on the determined payload (from PassengerCoordinator/MaintenanceCoordinator)
     and wind data (from WeatherAnalyst), **calculate the optimal cruising altitude(s) for maximum fuel efficiency, considering the latest wind data (jet stream).
     Recommend specific Mach numbers or airspeeds for different flight segments (climb, cruise, descent) to minimize fuel burn.** Also provide the estimated fuel burn for the transatlantic primary route.
     You may use available tools via the Admin for calculations and data retrieval, including the 'ask_deepseek_flight_ai' tool for complex performance calculations.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

contingency_planner_agent = ConversableAgent(
    name="ContingencyPlanner",
    system_message="""You are an Emergency and Contingency Flight Planner. Develop a robust contingency plan for the KORD to EDDF flight.
    Include at least three suitable alternate airports (e.g., Gander, Reykjavik, Shannon, Amsterdam) and basic diversion procedures for a transatlantic crossing.
    **Include considerations for fuel-efficient diversion profiles if an alternate landing is required.**
    Use tools via the Admin if needed to find airport information or weather for alternates.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

maintenance_coordinator_agent = ConversableAgent(
    name="MaintenanceCoordinator",
    system_message="""You are an Aircraft Maintenance and Readiness Coordinator for a Boeing 777-300ER (Tail N777UA).
    Assess aircraft maintenance status and identify any relevant limitations or weight considerations for the flight on June 7, 2025.
    Use the 'search_aviation_database' tool via the Admin to get maintenance records.""",
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

passenger_coordinator_agent = ConversableAgent(
    name="PassengerCoordinator",
    system_message="""You are a Passenger and Cargo Load Estimator. Estimate the total weight of passengers, baggage, and cargo
    for the KORD-EDDF flight on June 7, 2025, to determine the flight payload.
    Use the 'search_aviation_database' tool via the Admin to get passenger manifest information.
    Provide the estimated number of adult/child passengers, estimated baggage/cargo weight, and the total estimated payload (in lbs or kg)
    to the PerformanceSpecialist and FlightPlanner.
    """,
    llm_config={"config_list": config_list}, # Uses the DeepSeek-only config_list
    human_input_mode="NEVER",
)

# 5. Create and Initiate the Group Chat
from autogen import GroupChat, GroupChatManager

groupchat = GroupChat(
    agents=[
        user_proxy,
        flight_planner_agent,
        weather_forecaster_agent,
        airspace_regulator_agent,
        aircraft_performance_analyst_agent,
        contingency_planner_agent,
        maintenance_coordinator_agent,
        passenger_coordinator_agent
        ],
    messages=[],
    max_round=30,
    speaker_selection_method="auto",
    allow_repeat_speaker=False
)

# The manager also needs the config_list
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

print("\nStarting the AutoGen Flight Planning Collaboration using only DeepSeek models...")

# Check if models were configured before starting the chat
# This check is now redundant due to the earlier fatal error check,
# but keeping it doesn't hurt.
if config_list:
    user_proxy.initiate_chat(
        manager,
        message="""Generate a detailed, fuel-efficient flight plan for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
        The plan must include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
        the most fuel-efficient North Atlantic Track (NAT) for eastbound traffic on June 7, 2025 (including all oceanic waypoints, entry/exit points),
        recommended step climbs/Mach/airspeed for fuel efficiency, and at least two alternate airports.
        Collaborate as a team to gather all necessary information (weather, regulations, maintenance, payload) and calculate performance before the FlightPlanner generates the final output.
        The final output should be a markdown formatted document.
        Ensure the final message clearly states 'TERMINATE' at the end."""
    )
else:
    # This else block will only be reached if you comment out the `raise ValueError` line
    print("Chat not initiated because no DeepSeek models were configured due to missing API key (as checked earlier).")


print("\n\n########################")
print("## AutoGen Conversation Finished ##")
print("########################\n")

Attempted to get DeepSeek API key from Colab userdata.
DeepSeek API Key successfully retrieved (starts with: sk-e...)

Starting the AutoGen Flight Planning Collaboration using only DeepSeek models...
Admin (to chat_manager):

Generate a detailed, fuel-efficient flight plan for a Boeing 777-300ER from KORD to EDDF, departing 20:00Z on June 7, 2025.
        The plan must include: optimal cruising altitude, estimated time en route, optimized fuel requirements,
        the most fuel-efficient North Atlantic Track (NAT) for eastbound traffic on June 7, 2025 (including all oceanic waypoints, entry/exit points),
        recommended step climbs/Mach/airspeed for fuel efficiency, and at least two alternate airports.
        Collaborate as a team to gather all necessary information (weather, regulations, maintenance, payload) and calculate performance before the FlightPlanner generates the final output.
        The final output should be a markdown formatted document.
        Ensure the final me

In [3]:
from google.colab import userdata
from litellm import completion
import os

deepseek_api_key_test = userdata.get('DEEPSEEK_API_KEY')
#print(f"Test Key: {deepseek_api_key_test}")

try:
    response = completion(
        model="deepseek/deepseek-reasoner",
        messages=[{"role": "user", "content": "hello"}],
        api_key=deepseek_api_key_test # Explicitly pass the key
    )
    print("Test call successful!")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Test call failed: {e}")

Test call successful!
Hello! 😊 How can I help you today?
